In [2]:
import numpy as np
import cv2

In [11]:
data = np.load('intrinsics.npz')
Lcam = data["Lcam"]
print(Lcam)

[[ 2.12949203e+03  1.55132281e+02  8.20439554e+01  1.41050036e+04]
 [-1.55062713e+02  2.13326434e+03  9.83929178e+01  6.94311769e+03]
 [ 5.47928716e-02 -2.09287937e-02  9.98278381e-01  9.97801819e+01]]


In [ ]:

Lcam = np.array([[ 2.12949203e+03  1.55132281e+02  8.20439554e+01  1.41050036e+04]
 [-1.55062713e+02  2.13326434e+03  9.83929178e+01  6.94311769e+03]
 [ 5.47928716e-02 -2.09287937e-02  9.98278381e-01  9.97801819e+01]])


In [8]:
grid_counts = (7, 9)
# grid_lengths = (7., 9.)
side_lengths = (7.4/8, 9.7/11)
# side_lengths = (grid_lengths[0]/grid_counts[0], grid_lengths[1]/grid_counts[1])

def pxl_to_board(px, py, Z=0):
    X = np.linalg.inv(np.hstack((Lcam[:,0:2] ,np.array([[-1*px],[-1*py],[-1]])))).dot((-Z*Lcam[:,2]-Lcam[:,3]))
    return X[:2]

center_board = pxl_to_board(0,0)

def pxl_to_pose(px, py):
    X = pxl_to_board(px, py) - center_board
#     X = np.linalg.inv(np.hstack((Lcam[:,0:2] ,np.array([[-1*px],[-1*py],[-1]])))).dot((-Z*Lcam[:,2]-Lcam[:,3]))
    return [X[0]*side_lengths[0], X[1]*side_lengths[1]]


In [10]:
pxl_to_pose(1,0)

[0.0429980770910829, 0.002979538857407858]

In [ ]:
resolution = (1080, 720)
x_res = resolution[0]

# translations = []
x_pxls = np.arange(x_res)
x_meters = [pxl_to_pose(px, 0) for px in x_pxls]
angles = [np.atan2(x_meter, height) for x_meter in x_meters]

# np.interp 

def x_pxl_to_angle(x_pxl):
    return angles[x_pxl]


In [ ]:
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

grid_size
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
image_file = 'checkerboard.png'
img = cv.imread(fname)
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
# Find the chess board corners
ret, corners = cv.findChessboardCorners(gray, (7,6), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners)
        # Draw and display the corners
        cv.drawChessboardCorners(img, (7,6), corners2, ret)
        cv.imshow('img', img)
        cv.waitKey(500)
cv.destroyAllWindows()

In [ ]:
import numpy as np
import cv2

CHECKERBOARD = (6, 9)


# stop the iteration when specified
# accuracy, epsilon, is reached or
# specified number of iterations are completed.
criteria = (cv2.TERM_CRITERIA_EPS +
            cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)


# Vector for 3D points
obj_points = []

# Vector for 2D points
img_points = []


#  3D points real world coordinates
objectp3d = np.zeros((1, CHECKERBOARD[0]
                      * CHECKERBOARD[1],
                      3), np.float32)
objectp3d[0, :, :2] = np.mgrid[0:CHECKERBOARD[0],
                               0:CHECKERBOARD[1]].T.reshape(-1, 2)

 
filename = "a.jpg"
image = cv2.imread(filename)
grayColor = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Find the chess board corners
# If desired number of corners are
# found in the image then ret = true
ret, corners = cv2.findChessboardCorners(
                grayColor, CHECKERBOARD,
                cv2.CALIB_CB_ADAPTIVE_THRESH
                + cv2.CALIB_CB_FAST_CHECK +
                cv2.CALIB_CB_NORMALIZE_IMAGE)

# If desired number of corners can be detected then,
# refine the pixel coordinates and display
# them on the images of checker board
if ret == True:
    obj_points.append(objectp3d)

    # Refining pixel coordinates
    # for given 2d points.
    corners2 = cv2.cornerSubPix(
        grayColor, corners, (11, 11), (-1, -1), criteria)

    img_points.append(corners2)

    # Draw and display the corners
    image = cv2.drawChessboardCorners(image,
                                        CHECKERBOARD,
                                        corners2, ret)
else:
    print("Failed")

cv2.imshow('img', image)
cv2.waitKey(0)

cv2.destroyAllWindows()

ret, matrix, distortion, r_vecs, t_vecs = cv2.calibrateCamera(
    obj_points, img_points, grayColor.shape[::-1], None, None)

Lcam = matrix.dot(np.hstack((cv2.Rodrigues(r_vecs[0])[0],t_vecs[0])))

np.save("intrinsics_.npy",
    ret=ret, matrix=matrix, distortion=distortion,
    r_vecs=r_vecs, t_vecs=t_vecs, Lcam=Lcam,
    obj_points=obj_points, img_points=img_points)